In [4]:
import torch
from torchvision import transforms
from PIL import Image

In [5]:
from nuscenes.can_bus.can_bus_api import NuScenesCanBus
from nuscenes.nuscenes import NuScenes
PATH = '/Users/jonathanmorris/Downloads/v1.0-mini'

nusc = NuScenes(version='v1.0-mini', dataroot=PATH, verbose=True)
nusc_can = NuScenesCanBus(dataroot=PATH)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.481 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [6]:
nusc.list_scenes()

scene-0061, Parked truck, construction, intersectio... [18-07-24 03:28:47]   19s, singapore-onenorth, #anns:4622
scene-0103, Many peds right, wait for turning car, ... [18-08-01 19:26:43]   19s, boston-seaport, #anns:2046
scene-0655, Parking lot, parked cars, jaywalker, be... [18-08-27 15:51:32]   20s, boston-seaport, #anns:2332
scene-0553, Wait at intersection, bicycle, large tr... [18-08-28 20:48:16]   20s, boston-seaport, #anns:1950
scene-0757, Arrive at busy intersection, bus, wait ... [18-08-30 19:25:08]   20s, boston-seaport, #anns:592
scene-0796, Scooter, peds on sidewalk, bus, cars, t... [18-10-02 02:52:24]   20s, singapore-queensto, #anns:708
scene-0916, Parking lot, bicycle rack, parked bicyc... [18-10-08 07:37:13]   20s, singapore-queensto, #anns:2387
scene-1077, Night, big street, bus stop, high speed... [18-11-21 11:39:27]   20s, singapore-hollandv, #anns:890
scene-1094, Night, after rain, many peds, PMD, ped ... [18-11-21 11:47:27]   19s, singapore-hollandv, #anns:1762
sc

In [7]:
my_scene = nusc.scene[0]
my_scene_can = nusc_can.get_messages(my_scene['name'], 'zoesensors')
my_scene_can[0]

{'brake_sensor': 0.18794798851013184,
 'steering_sensor': 0.18827444314956665,
 'throttle_sensor': 0.12093144655227661,
 'utime': 1532402927649559}

In [8]:
first_sample_token = my_scene['first_sample_token']
my_sample = nusc.get('sample', first_sample_token)

In [9]:
def get_closest_can(time, can_objects):
    closest = can_objects[0]
    for object in can_objects:
        if object["utime"] - time < closest["utime"] - time:
            closest = object
    return closest

def normalize_can(can_obj):
    # These values are from here: https://github.com/nutonomy/nuscenes-devkit/blob/master/python-sdk/nuscenes/can_bus/README.md#zoe-sensors
    min_breaking = 0.166
    max_breaking = 0.631
    min_steering = 0.176
    max_steering = 0.252
    min_throttle = 0.105
    max_throttle = 0.411

    # convert values from 0-1
    can_obj["brake_sensor"] = (can_obj["brake_sensor"] - min_breaking) / (max_breaking - min_breaking)
    can_obj["steering_sensor"] = (can_obj["steering_sensor"] - min_steering) / (max_steering - min_steering)
    can_obj["throttle_sensor"] = (can_obj["throttle_sensor"] - min_throttle) / (max_throttle - min_throttle)

    # reduce precision
    can_obj["brake_sensor"] = round(can_obj["brake_sensor"], 2)
    can_obj["steering_sensor"] = round(can_obj["steering_sensor"], 2)
    can_obj["throttle_sensor"] = round(can_obj["throttle_sensor"], 2)
    
    

In [10]:
can_data = get_closest_can(my_sample['timestamp'], my_scene_can)
print("Before: " + str(can_data))
normalize_can(can_data)
print("After: " + str(can_data))

Before: {'brake_sensor': 0.18794798851013184, 'steering_sensor': 0.18827444314956665, 'throttle_sensor': 0.12093144655227661, 'utime': 1532402927649559}
After: {'brake_sensor': 0.05, 'steering_sensor': 0.16, 'throttle_sensor': 0.05, 'utime': 1532402927649559}


In [11]:
sensor = 'CAM_FRONT'
cam_front_data = nusc.get('sample_data', my_sample['data'][sensor])
current_image_path = PATH+"/"+cam_front_data['filename']

In [12]:
img = Image.open(current_image_path)
convert_tensor = transforms.ToTensor()
convert_tensor(img)

tensor([[[0.1216, 0.1176, 0.1098,  ..., 0.4863, 0.4863, 0.4863],
         [0.1176, 0.1176, 0.1137,  ..., 0.4863, 0.4863, 0.4863],
         [0.0980, 0.1020, 0.1020,  ..., 0.4863, 0.4863, 0.4863],
         ...,
         [0.1020, 0.2431, 0.2627,  ..., 0.4000, 0.4039, 0.4039],
         [0.1020, 0.2431, 0.2627,  ..., 0.3961, 0.3961, 0.4000],
         [0.1020, 0.2431, 0.2627,  ..., 0.3922, 0.3961, 0.3961]],

        [[0.0863, 0.0824, 0.0863,  ..., 0.5412, 0.5412, 0.5412],
         [0.0824, 0.0824, 0.0902,  ..., 0.5412, 0.5412, 0.5412],
         [0.0627, 0.0667, 0.0784,  ..., 0.5412, 0.5412, 0.5412],
         ...,
         [0.1020, 0.2431, 0.2627,  ..., 0.4000, 0.4039, 0.4039],
         [0.1020, 0.2431, 0.2627,  ..., 0.3961, 0.3961, 0.4000],
         [0.1020, 0.2431, 0.2627,  ..., 0.3922, 0.3961, 0.3961]],

        [[0.0980, 0.0941, 0.1020,  ..., 0.5922, 0.5922, 0.5922],
         [0.0941, 0.0941, 0.1059,  ..., 0.5922, 0.5922, 0.5922],
         [0.0667, 0.0706, 0.0863,  ..., 0.5922, 0.5922, 0.